## Кейс 3

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import copy
from datetime import date

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', 80)
pd.set_option('display.max_rows', 70)
pd.get_option('display.precision', 2)
pd.set_option('display.float_format', '{:.2f}'.format)

In [3]:
df = pd.read_csv('/content/drive/MyDrive/dataset_task3.csv')

In [4]:
df.head(3)

,Дата,Кампания,№ Кампании,Показы,Ср. позиция показов
0,янв 2022,o-a_corporate_perform_god_20220100016_context_search_brand_srb_yxprrko,69149644,4442,"1,1"
1,янв 2022,o-a_corporate_perform_god_20220100016_context_search_brand_url_yxprrko,69149675,2692,"1,04"
2,янв 2022,o-a_corporate_perform_god_20220100016_context_search_brand_cch_yxprrko,69149701,1382,"1,06"


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Дата                 64 non-null     object
 1   Кампания             64 non-null     object
 2   № Кампании           64 non-null     int64 
 3   Показы               64 non-null     int64 
 4   Ср. позиция показов  64 non-null     object
dtypes: int64(2), object(3)
memory usage: 2.6+ KB


In [6]:
df.columns = ['dt',
              'marketing_campaign',
              'campaign_number',
              'ad_impressions', 
              'avg_impression_position']

In [7]:
# В Ubuntu установлена локаль en_US.UTF-8, а не ru_RU, 
# поэтому парсинг даты проводится с помощью самописной функции
def func_dt_transform(dt_str:str)->date:
  """Функция трансформирует строку в дату"""
  month_names = ['', 'янв', 'фев', 'мар', 'апр', 'май', 'июн','июл', 'авг',  \
                 'сен', 'окт', 'ноя', 'дек']
  try:
    m_name,y_number = dt_str.split(" ")
    current_dt_str = str(month_names.index(m_name)) + "-" + str(y_number)
    return dt.datetime.strptime(current_dt_str,'%m-%Y').date()
  except Exception:
    print('Невозможно трасформировать строку в дату!')

In [8]:
def func_campaign_parsing(marketing_campaign_str:str, 
                          parameter:str='campaign_name')->str:
  """Функция выделяет параметр кампании и ее название из строки"""
  marketing_campaign_list = marketing_campaign_str.split("_")
  try:
    if parameter =='campaign_name':
      return "_".join(marketing_campaign_list[-2:])
    else:
      return marketing_campaign_list[-3:-2][0]
  except Exception:
     print('Невозможно выделить значения из строки!')


In [9]:
# Трансформируем строки в даты
df['dt'] = df['dt'].apply(lambda date: func_dt_transform(date))
# Трасформируем средние позиции показов рекламных объявлений в числа
df['avg_impression_position'] = df['avg_impression_position'] \
                               .apply(lambda val: float(str.replace(val,',','.')))
# Выделяем тип кампании и ее название из "сырой" строки
df[['campaign_type', 'campaign_name']] = df['marketing_campaign'] \
                .apply(lambda x: pd.Series({'campaign_type':func_campaign_parsing(x,'type'),  \
                                            'campaign_name':func_campaign_parsing(x)}))
# Удаление технического столбца после парсинга
df = df.drop('marketing_campaign', axis=1)
# Меняем порядок столбцов вывода датафрейма
columns_titles = ['dt', 
                  'campaign_type', 
                  'campaign_name', 
                  'campaign_number',
                  'ad_impressions',
                  'avg_impression_position']
df = df.reindex(columns=columns_titles)

In [10]:
df = df.sort_values(by=['campaign_type','avg_impression_position'],ascending=[True,True])

In [11]:
df.head()

,dt,campaign_type,campaign_name,campaign_number,ad_impressions,avg_impression_position
7,2022-01-01,brand,dvb_yxprrko,69149838,443,1.02
1,2022-01-01,brand,url_yxprrko,69149675,2692,1.04
10,2022-01-01,brand,uzb_yxprrko,69149921,2390,1.04
5,2022-01-01,brand,bkl_yxprrko,69149785,564,1.05
2,2022-01-01,brand,cch_yxprrko,69149701,1382,1.06


In [12]:
df.to_csv('/content/drive/MyDrive/marketing_campaign.csv', index=False) 

Расчет запрашиваемых показателей я решил провести на BI-платформе по следующим причинам:
*  презентация позволяет выполнять гибкую верстку согласно корпоративным стандартам, но теряется возможность быстрого обновления данных;
*  расчеты в jupyter-ноутбуках позволяют гибко работать с датасетами, но большинство библиотек для построения графиков и диаграмм значительно ограничены по функционалу;
*  инструменты подобные Apache SuperSet требуют вспомогательной платформы для демонстрации итогового дашборда заказчику.

Ввиду вышеизложенного Python будет выполнять роль лишь etl-инструмента)